In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

In [155]:
df=pd.read_csv('final_railway_dataset.csv')

In [156]:
df=df.drop('Transaction ID',axis=1)

In [182]:
df.head()

,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,Arrival Destination,Journey Status,Reason for Delay,...,Arrival Minute_sin,Arrival Minute_cos,Arrival Second_sin,Arrival Second_cos,Actual Arrival Hour_sin,Actual Arrival Hour_cos,Actual Arrival Minute_sin,Actual Arrival Minute_cos,Actual Arrival Second_sin,Actual Arrival Second_cos
0,0,0,0,1,0,43,6,12,2,0,...,5.665539e-16,-1.000000e+00,0.0,1.0,-0.258819,-9.659258e-01,5.665539e-16,-1.000000e+00,0.0,1.0
1,1,1,0,1,0,23,5,31,1,1,...,-5.000000e-01,-8.660254e-01,0.0,1.0,0.258819,-9.659258e-01,-8.660254e-01,-5.000000e-01,0.0,1.0
2,0,1,2,1,0,3,3,18,2,0,...,-1.000000e+00,-1.836970e-16,0.0,1.0,-1.000000,-1.836970e-16,-1.000000e+00,-1.836970e-16,0.0,1.0
3,1,1,2,1,0,13,6,23,2,0,...,5.665539e-16,-1.000000e+00,0.0,1.0,-0.500000,8.660254e-01,5.665539e-16,-1.000000e+00,0.0,1.0
4,0,0,2,1,0,76,3,13,2,0,...,0.000000e+00,1.000000e+00,0.0,1.0,-0.965926,2.588190e-01,0.000000e+00,1.000000e+00,0.0,1.0


In [164]:
df.shape

(31653, 46)

In [159]:
date=['Date of Purchase','Date of Journey']
for i in date:
    df[i] = pd.to_datetime(df[i])
    df[i[8:]+" Year"] = df[i].dt.year
    df[i[8:]+" Month"] = df[i].dt.month
    df[i[8:]+" Day"] = df[i].dt.day
    df[i[8:]+" DayOfWeek"] = df[i].dt.dayofweek 
    df[i[8:]+" IsWeekend"] = df[i[8:]+" DayOfWeek"].isin([5, 6]).astype(int)
    df = df.drop(columns=[i])

In [160]:
df=df.rename(columns={'Time of Purchase':'Purchase Time'})

In [161]:
df['Actual Arrival Time'].replace("didn't Arrive",'00:00:00',inplace=True)

C:\Users\ahmed\AppData\Local\Temp\ipykernel_5748\2106156916.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Actual Arrival Time'].replace("didn't Arrive",'00:00:00',inplace=True)


In [162]:
indexs=df[df['Actual Arrival Time']=='00:00:00'].index

In [163]:
time=['Purchase Time','Departure Time','Arrival Time','Actual Arrival Time']
for i in time:
    df[i] = pd.to_datetime(df[i], format="%H:%M:%S").dt.time
    df[i[:-5]+" Hour"] = pd.to_datetime(df[i], format="%H:%M:%S").dt.hour
    df[i[:-5]+" Minute"] = pd.to_datetime(df[i], format="%H:%M:%S").dt.minute
    df[i[:-5]+" Second"] = pd.to_datetime(df[i], format="%H:%M:%S").dt.second
    
    df = df.drop(columns=[i])
    
    df[i[:-5]+" Hour_sin"] = np.sin(2 * np.pi * df[i[:-5]+" Hour"] / 24)
    df[i[:-5]+" Hour_cos"] = np.cos(2 * np.pi * df[i[:-5]+" Hour"] / 24)
    
    df[i[:-5]+" Minute_sin"] = np.sin(2 * np.pi * df[i[:-5]+" Minute"] / 60)
    df[i[:-5]+" Minute_cos"] = np.cos(2 * np.pi * df[i[:-5]+" Minute"] / 60)
    
    df[i[:-5]+" Second_sin"] = np.sin(2 * np.pi * df[i[:-5]+" Second"] / 60)
    df[i[:-5]+" Second_cos"] = np.cos(2 * np.pi * df[i[:-5]+" Second"] / 60)
    
    
    df = df.drop(columns=[i[:-5]+" Hour",i[:-5]+" Minute",i[:-5]+" Second"])

In [170]:
vals=df['Refund Request'].unique()
for i in range(vals.shape[0]):
    df['Refund Request'].replace(vals[i],i,inplace=True)

C:\Users\ahmed\AppData\Local\Temp\ipykernel_5748\1399502229.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Refund Request'].replace(vals[i],i,inplace=True)
C:\Users\ahmed\AppData\Local\Temp\ipykernel_5748\1399502229.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Refund Request'].re

In [174]:
df.select_dtypes(exclude=['number']).columns

Index(['Purchase Type', 'Payment Method', 'Railcard', 'Ticket Class',
       'Ticket Type', 'Departure Station', 'Arrival Destination',
       'Journey Status', 'Reason for Delay'],
      dtype='object')

In [181]:
le=LabelEncoder()
columns=df.select_dtypes(exclude=['number']).columns
for col in columns:
    df[col]=le.fit_transform(df[col])

In [192]:
df['Refund Request'].value_counts()

Refund Request
0    30535
1     1118
Name: count, dtype: int64

In [185]:
df.columns

Index(['Purchase Type', 'Payment Method', 'Railcard', 'Ticket Class',
       'Ticket Type', 'Price', 'Departure Station', 'Arrival Destination',
       'Journey Status', 'Reason for Delay', 'Refund Request',
       'Original Price', 'Purchase Year', 'Purchase Month', 'Purchase Day',
       'Purchase DayOfWeek', 'Purchase IsWeekend', 'Journey Year',
       'Journey Month', 'Journey Day', 'Journey DayOfWeek',
       'Journey IsWeekend', 'Purchase Hour_sin', 'Purchase Hour_cos',
       'Purchase Minute_sin', 'Purchase Minute_cos', 'Purchase Second_sin',
       'Purchase Second_cos', 'Departure Hour_sin', 'Departure Hour_cos',
       'Departure Minute_sin', 'Departure Minute_cos', 'Departure Second_sin',
       'Departure Second_cos', 'Arrival Hour_sin', 'Arrival Hour_cos',
       'Arrival Minute_sin', 'Arrival Minute_cos', 'Arrival Second_sin',
       'Arrival Second_cos', 'Actual Arrival Hour_sin',
       'Actual Arrival Hour_cos', 'Actual Arrival Minute_sin',
       'Actual Arrival Minu

In [189]:
prediction_df=df[[
    'Purchase Type', 'Payment Method', 'Railcard', 'Ticket Class',
    'Ticket Type', 'Price', 'Original Price', 'Purchase Year', 
    'Purchase Month', 'Purchase Day', 'Purchase DayOfWeek', 'Purchase IsWeekend',
    'Journey Year', 'Journey Month', 'Journey Day', 'Journey DayOfWeek', 
    'Journey IsWeekend', 'Departure Station', 'Arrival Destination', 'Journey Status', 
    'Reason for Delay', 'Purchase Hour_sin', 'Purchase Hour_cos', 'Purchase Minute_sin',
    'Purchase Minute_cos', 'Purchase Second_sin', 'Purchase Second_cos', 
    'Departure Hour_sin', 'Departure Hour_cos', 'Departure Minute_sin', 'Departure Minute_cos',
    'Departure Second_sin', 'Departure Second_cos', 'Arrival Hour_sin', 'Arrival Hour_cos',
    'Arrival Minute_sin', 'Arrival Minute_cos', 'Arrival Second_sin', 'Arrival Second_cos',
    'Actual Arrival Hour_sin', 'Actual Arrival Hour_cos', 'Actual Arrival Minute_sin',
    'Actual Arrival Minute_cos', 'Actual Arrival Second_sin', 'Actual Arrival Second_cos']]

In [190]:
prediction_df.head()

,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Original Price,Purchase Year,Purchase Month,Purchase Day,...,Arrival Minute_sin,Arrival Minute_cos,Arrival Second_sin,Arrival Second_cos,Actual Arrival Hour_sin,Actual Arrival Hour_cos,Actual Arrival Minute_sin,Actual Arrival Minute_cos,Actual Arrival Second_sin,Actual Arrival Second_cos
0,0,0,0,1,0,43,128.36,2023,12,8,...,5.665539e-16,-1.000000e+00,0.0,1.0,-0.258819,-9.659258e-01,5.665539e-16,-1.000000e+00,0.0,1.0
1,1,1,0,1,0,23,68.66,2023,12,16,...,-5.000000e-01,-8.660254e-01,0.0,1.0,0.258819,-9.659258e-01,-8.660254e-01,-5.000000e-01,0.0,1.0
2,0,1,2,1,0,3,6.00,2023,12,19,...,-1.000000e+00,-1.836970e-16,0.0,1.0,-1.000000,-1.836970e-16,-1.000000e+00,-1.836970e-16,0.0,1.0
3,1,1,2,1,0,13,26.00,2023,12,20,...,5.665539e-16,-1.000000e+00,0.0,1.0,-0.500000,8.660254e-01,5.665539e-16,-1.000000e+00,0.0,1.0
4,0,0,2,1,0,76,152.00,2023,12,27,...,0.000000e+00,1.000000e+00,0.0,1.0,-0.965926,2.588190e-01,0.000000e+00,1.000000e+00,0.0,1.0


In [207]:
df_minority = df[df['Refund Request'] == 1]
df_majority = df[df['Refund Request'] == 0] 

df_majority_downsampled = resample(df_majority,
                                   replace=False,   
                                   n_samples=len(df_minority),
                                   random_state=42)


df_balanced = pd.concat([df_majority_downsampled, df_minority])

df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [208]:
df_balanced['Refund Request']

0       1
1       0
2       1
3       0
4       1
       ..
2231    1
2232    0
2233    1
2234    1
2235    0
Name: Refund Request, Length: 2236, dtype: int64

In [199]:
df_balanced.shape

(2236, 46)

In [201]:
scaler = StandardScaler()

columns_to_scale = ['Price', 'Purchase Hour_sin', 'Purchase Hour_cos', 
                    'Purchase Minute_sin', 'Purchase Minute_cos', 'Purchase Second_sin', 'Purchase Second_cos',
                    'Departure Hour_sin', 'Departure Hour_cos', 'Departure Minute_sin', 'Departure Minute_cos', 
                    'Departure Second_sin', 'Departure Second_cos', 'Arrival Hour_sin', 'Arrival Hour_cos',
                    'Arrival Minute_sin', 'Arrival Minute_cos', 'Arrival Second_sin', 'Arrival Second_cos',
                    'Actual Arrival Hour_sin', 'Actual Arrival Hour_cos', 'Actual Arrival Minute_sin', 
                    'Actual Arrival Minute_cos', 'Actual Arrival Second_sin', 'Actual Arrival Second_cos']

df_balanced[columns_to_scale] = scaler.fit_transform(df_balanced[columns_to_scale])

In [202]:
df_balanced.head()

,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,Arrival Destination,Journey Status,Reason for Delay,...,Arrival Minute_sin,Arrival Minute_cos,Arrival Second_sin,Arrival Second_cos,Actual Arrival Hour_sin,Actual Arrival Hour_cos,Actual Arrival Minute_sin,Actual Arrival Minute_cos,Actual Arrival Second_sin,Actual Arrival Second_cos
0,1,1,0,1,0,-0.728958,4,0,1,2,...,1.244730,-0.703667,0.0,0.0,-1.310340,-0.581489,-1.201801,0.462217,0.0,0.0
1,0,1,2,1,0,0.195236,5,31,2,0,...,0.766478,1.445570,0.0,0.0,-1.310340,0.113687,0.783679,0.744068,0.0,0.0
2,0,1,3,1,0,-0.520269,9,1,1,1,...,0.113173,-1.490343,0.0,0.0,-0.898486,-1.183529,-1.473955,0.086605,0.0,0.0
3,1,0,2,0,0,0.702052,4,0,2,0,...,-1.018384,0.869684,0.0,0.0,1.763783,-0.581489,-1.398059,0.220109,0.0,0.0
4,0,1,2,1,2,-0.550082,4,0,0,6,...,-0.540132,-1.279554,0.0,0.0,0.226721,1.109077,-0.014893,0.935850,0.0,0.0


In [204]:
df_balanced.to_csv('preprocessed_dataset.csv',index=False)